In [ ]:
%matplotlib inline

In [ ]:
from os import environ
environ["CUDA_VISIBLE_DEVICES"] = "0"

from pathlib import Path
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from bliss.encoder.encoder import Encoder
from bliss.catalog import TileCatalog
from matplotlib import pyplot as plt

In [ ]:
from hydra import initialize, compose
from hydra.utils import instantiate

with initialize(config_path=".", version_base=None):
    overrides = {
        "predict.weight_save_path=/home/regier/bliss_output/jul25_toy_example_10_percent/version_0/checkpoints/best_encoder.ckpt",
        "decoder.with_noise=true",
        "decoder.with_dither=false",
        "encoder.predict_mode_not_samples=false",
        "train.trainer.logger=false",
        "train.trainer.max_epochs=0",
        "+train.trainer.num_sanity_val_steps=0",
        "cached_simulator.num_workers=0",
        "cached_simulator.splits=0:80/0:90/99:100",
#        "paths.cached_data=/data/scratch/regier/toy_example"
    }
    cfg = compose("toy_example", overrides)

decoder = instantiate(cfg.simulator.decoder)

torch.set_grad_enabled(False)

trainer = instantiate(cfg.train.trainer)

data_source = instantiate(cfg.train.data_source)
data_source.setup("fit")
data_source.setup("test")

encoder = instantiate(cfg.encoder).cuda()
encoder.eval()
state_dict = torch.load(cfg.predict.weight_save_path)["state_dict"]
encoder.load_state_dict(state_dict);

## One bright (easily detected) star at various positions

Compare samples with independent and dependent tiling

In [ ]:
n = 50
n_sources = torch.zeros(n, 20, 20, dtype=torch.long)
n_sources[:, 11, 10] = 1

locs = torch.ones(n, 20, 20, 1, 2) * 0.5
locs[:, 11, 10, 0, 0] = 0.0

source_type = torch.zeros(n, 20, 20, 1, 1, dtype=torch.long)

true_catalog_dict = {
    "n_sources": n_sources,
    "source_type": source_type,
    "locs": locs,
    "star_fluxes": torch.ones(n, 20, 20, 1, 1) * 8,
    "galaxy_fluxes": torch.ones(n, 20, 20, 1, 1),
}
true_catalog = TileCatalog(true_catalog_dict)

In [ ]:
images, psf_params = decoder.render_images(true_catalog)
images = images[:, 2:3]
psf_params = psf_params[:, 2:3]

In [ ]:
# plot a coadd image
plt.imshow(images[3].sum(0).numpy())
plt.colorbar();

In [ ]:
batch = {"images": images.cuda(), "psf_params": psf_params.cuda()}

# comparing joint and marginal

In [ ]:
mask = torch.zeros_like(true_catalog_dict["n_sources"].cuda())
context1 = encoder.make_context(None, mask)
x_features = encoder.get_features(batch)
x_cat_marginal = encoder.catalog_net(x_features, context1)
on_probs = x_cat_marginal[..., 0].sigmoid()
on_probs[:, 10:12, 10].sum(1).mean(0)

In [ ]:
joint_cats = encoder.sample(batch)
(joint_cats["n_sources"][:, 9:11, 9].sum(1) > 0).float().mean()

conclusions
 * In this setting, with one bright star very close to the border, even the *expected number of sources* by marginal isn't well calibrated. It underestimates by at least 5% for a variety of fluxes that are well above the detection limit.
 * Joint is essentially always right is expectation (and in each posterior sample). Though one pixel probability may be miscalibrated, the other makes up for it.